In [14]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd

RANDOM_SEED = 42

2022-10-05 12:21:56.779121: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
data_path = './data/data.csv'
model_path = './model/model.keras'

In [17]:
df = pd.read_csv(data_path)
df

,label,T-31_P0x,T-31_P0y,T-31_P0z,T-31_P1x,T-31_P1y,T-31_P1z,T-31_P2x,T-31_P2y,T-31_P2z,...,T-0_P17z,T-0_P18x,T-0_P18y,T-0_P18z,T-0_P19x,T-0_P19y,T-0_P19z,T-0_P20x,T-0_P20y,T-0_P20z
0,1.0,0.019263,0.048485,1.879826e-10,0.021847,0.045686,-0.000466,0.023458,0.040864,-0.000754,...,-0.002054,0.018095,0.031726,-0.002793,0.018241,0.029022,-0.003340,0.018489,0.026397,-0.003745
1,1.0,0.019556,0.050036,8.835470e-11,0.022346,0.047041,-0.000667,0.024055,0.042453,-0.001244,...,-0.002287,0.014547,0.033535,-0.002999,0.013894,0.031098,-0.003538,0.013433,0.028690,-0.003940
2,1.0,0.018768,0.048917,1.314620e-10,0.021628,0.046195,-0.000730,0.023458,0.041890,-0.001376,...,-0.002788,0.014871,0.034806,-0.003463,0.014359,0.032344,-0.003894,0.014039,0.029862,-0.004246
3,1.0,0.018849,0.047390,1.854362e-10,0.020983,0.043579,-0.000378,0.022037,0.038595,-0.000825,...,-0.001994,0.017994,0.031706,-0.002658,0.018098,0.029269,-0.003122,0.018314,0.027038,-0.003465
4,1.0,0.019334,0.051645,4.650522e-10,0.021398,0.046072,-0.000526,0.021826,0.040004,-0.000989,...,-0.001346,0.019643,0.028969,-0.001886,0.019848,0.026893,-0.002317,0.020085,0.024965,-0.002617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1.0,0.022151,0.037677,1.280735e-10,0.023155,0.035753,-0.000519,0.023602,0.033257,-0.000775,...,-0.001082,0.019016,0.030055,-0.001498,0.018579,0.028691,-0.001809,0.018210,0.027519,-0.002030
97,1.0,0.024037,0.041068,1.575421e-10,0.025487,0.038887,-0.000349,0.026453,0.035843,-0.000711,...,-0.001396,0.023417,0.031495,-0.001898,0.023567,0.029826,-0.002217,0.023782,0.028206,-0.002442
98,1.0,0.022936,0.041987,3.705660e-10,0.024718,0.039654,-0.000188,0.025836,0.036859,-0.000468,...,-0.001188,0.017392,0.030498,-0.001571,0.016899,0.029146,-0.001833,0.016488,0.027837,-0.002016
99,1.0,0.023447,0.041801,1.503696e-10,0.025153,0.040201,-0.000395,0.026247,0.037418,-0.000702,...,-0.001180,0.018991,0.031989,-0.001618,0.018484,0.030477,-0.001962,0.018082,0.029125,-0.002216
